In [ ]:
import os
from pathlib import Path

### Documnet folder

In [ ]:
data_path = f"{Path.cwd().parent}/data_files/"
print(data_path)

In [ ]:
file_path = f"{data_path}attention.pdf" 
print(file_path)

In [ ]:
from docling.document_converter import DocumentConverter

In [ ]:
source = file_path
converter = DocumentConverter()
doc = converter.convert(source).document

In [ ]:
import json
print(json.dumps(doc.export_to_dict(), indent=2))

In [ ]:
print(doc.export_to_markdown()[:1000]) 

In [ ]:
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions,
    PictureDescriptionApiOptions
)
from docling.document_converter import DocumentConverter, PdfFormatOption
from docling_core.types.doc.document import ImageRefMode

def convert_with_image_annotation(input_doc_path):
    api_key = os.environ.get("OPENAI_API_KEY")
    model = "gpt-4.1-mini"
    picture_desc_api_option = PictureDescriptionApiOptions(
        url="https://api.openai.com/v1/chat/completions",
        prompt="Describe this image in sentences in a single paragraph.",
        params=dict(
                model=model,
            ),
        headers={
            "Authorization": "Bearer " + api_key,
        },
        timeout=60,
    )

    pipeline_options = PdfPipelineOptions(
        do_picture_description=True,
        picture_description_options=picture_desc_api_option,
        enable_remote_services=True,
        generate_picture_images=True,
        images_scale=2,
    )

    converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
    )
    conv_res = converter.convert(source=input_doc_path)
    return conv_res

In [ ]:
result = convert_with_image_annotation(file_path)

In [ ]:
print(result.document.export_to_markdown(mark_annotations = True, include_annotations=True))

In [ ]:
def export_function_md_with_image_ref(conv_res, output_path:str, replace_blank:str="_"):
    
    output_dir = Path(output_path)
    output_dir.mkdir(parents=True, exist_ok=True)
    doc_filename = conv_res.input.file.stem.replace(" ", replace_blank)

    # Save markdown with externally referenced pictures
    md_filename = output_dir / f"{doc_filename}-with-image-refs.md"
    conv_res.document.save_as_markdown(md_filename, image_mode=ImageRefMode.REFERENCED, include_annotations=True)

In [ ]:
output_path = f"{data_path}docling_output"

export_function_md_with_image_ref(result, output_path)